# McStas to NeXus for ODIN

This notebook converts data from a McStas simulation output (`.h5` filetype) to a NeXus file that uses a file for the Odin instrument (written by CODA) as a template for the geometry information.

It adds events to the `timepix3` detector and the `beam_monitor_3` monitor.

In [ ]:
import numpy as np
import scipp as sc
import scippnexus as sx
import h5py as h5
import shutil

In [ ]:
def load_mcstas_simulation_data(
    file_path,
    nevents,
    data_path="entry1/data/transmission_event_signal_dat_list_p_t_x_y_z_vx_vy_vz/events",
):
    nevents = int(nevents)
    with sx.File(file_path, "r") as f:
        # The name p_t_x_y_z_vx_vy_vz represents
        # probability, time of arrival, position(x, y, z) and velocity(vx, vy, vz).
        # The name also represents the order of each field in the table.
        # For example, probability is the first field, so data['dim_1', 0] is the probability.
        data = f[data_path][()].rename_dims({"dim_0": "event"})

    probabilities = data["dim_1", 0].copy()
    probabilities.unit = "dimensionless"
    time_of_arrival = data["dim_1", 1].copy()
    time_of_arrival.unit = "s"  # Hardcoded unit from the data.
    positions = data["dim_1", 2:5]
    # Units are hardcoded from the data.
    x_pos = positions["dim_1", 0].copy()
    x_pos.unit = "m"
    y_pos = positions["dim_1", 1].copy()
    y_pos.unit = "m"

    inds = np.random.choice(
        np.arange(data.sizes["event"]),
        nevents,
        p=probabilities.values / probabilities.values.sum(),
    )

    out = sc.DataArray(
        data=sc.array(
            dims=probabilities.dims,
            values=np.ones(nevents),
            unit="counts",
        ),
        coords={
            "time_of_arrival": sc.array(
                dims=time_of_arrival.dims,
                values=time_of_arrival.to(unit="us").values[inds],
                unit="us",
            ),
            "x": sc.array(
                dims=x_pos.dims,
                values=x_pos.values[inds],
                unit=x_pos.unit,
            ),
            "y": sc.array(
                dims=y_pos.dims,
                values=y_pos.values[inds],
                unit=y_pos.unit,
            ),
        },
    )
    return out

In [ ]:
def replace_dataset(entry, name, values):
    attrs = dict(entry[name].attrs)
    del entry[name]
    dset = entry.create_dataset(name, data=values)
    dset.attrs.update(attrs)


def mcstas_to_nexus(
    mcstas_data_file: str,
    template_nexus_file: str,
    outfile: str,
    nevents: float = 1e6,
    detector_entry_path: str | None = "entry/instrument/event_mode_detectors/timepix3",
    monitor_entry_path: str | None = "entry/instrument/beam_monitor_3",
):
    """
    Store the events from a McStas Odin simulation in a NeXus CODA file.

    Parameters
    ----------
    mcstas_data_file:
        Data file containing simulated McStas events.
    template_nexus_file:
        NeXus file containing geometry and instrument info, used as a template.
    outfile:
        Output file to be written.
    nevents:
        Number of events to have in the output file
        (events are sampled from the probabilities of the mcstas events).
    detector_entry_path:
        Location of the event detector in the nexus tree.
    monitor_entry_path:
        Location of the event monitor in the nexus tree.
    """
    # Find shape of detector panel
    with h5.File(template_nexus_file, "r") as f:
        shape = f[f"{detector_entry_path}/x_pixel_offset"].shape
        det_numbers = f[f"{detector_entry_path}/detector_number"][()]

    da = load_mcstas_simulation_data(mcstas_data_file, nevents=int(nevents))
    binned = da.bin(y=shape[0], x=shape[1]).rename_dims(y="dim_0", x="dim_1")

    toa = binned.bins.coords["time_of_arrival"].bins.concat().value

    # IMPORTANT! we need to sort the arrays below according to toa,
    # so that the event_index does not get messed up!
    event_id = sc.sort(
        (
            sc.bins_like(binned, sc.array(dims=binned.dims, values=det_numbers))
            .bins.concat()
            .value
        ),
        key=toa,
    )

    unit = "ns"
    period = (1.0 / sc.scalar(14.0, unit="Hz")).to(unit=unit)
    start = sc.datetime("2024-01-01T12:00:00.000000000")

    event_time_zero = sc.sort(
        (period * (toa.to(unit="ns", copy=False) // period)).to(dtype=int) + start,
        key=toa,
    )

    event_time_offset = sc.sort(toa % period.to(unit=toa.unit), key=toa)

    event_index = sc.DataArray(
        data=sc.ones_like(event_time_offset),
        coords={"event_time_zero": event_time_zero},
    ).group("event_time_zero")

    event_index = sc.cumsum(event_index.bins.size())
    event_index.values = np.concatenate([[0], event_index.values[:-1]])

    # Now edit the template file
    print(f"Writing {outfile} file")
    shutil.copyfile(template_nexus_file, outfile)
    f = h5.File(outfile, "r+")

    # Detector data
    if detector_entry_path is not None:
        # remove translation offset
        f[f"{detector_entry_path}/transformations/translation"].attrs["offset"] = (
            np.array([0, 0, 0], dtype="float32")
        )

        event_data = f[f"{detector_entry_path}/timepix3_events"]
        replace_dataset(event_data, name="event_id", values=event_id.values)
        replace_dataset(
            event_data,
            name="event_time_offset",
            values=event_time_offset.to(
                unit=event_data["event_time_offset"].attrs["units"], copy=False
            ).values,
        )
        replace_dataset(event_data, name="event_index", values=event_index.values)
        replace_dataset(
            event_data,
            name="event_time_zero",
            values=event_index.coords["event_time_zero"]
            .to(unit=event_data["event_time_zero"].attrs["units"], copy=False)
            .values.astype(int),
        )

    # Monitor data
    if monitor_entry_path is not None:
        monitor_data = f[f"{monitor_entry_path}/monitor_3_events"]
        replace_dataset(
            monitor_data, name="event_id", values=np.zeros_like(event_id.values)
        )
        replace_dataset(
            monitor_data,
            name="event_time_offset",
            values=event_time_offset.to(
                unit=monitor_data["event_time_offset"].attrs["units"], copy=False
            ).values,
        )
        replace_dataset(monitor_data, name="event_index", values=event_index.values)
        replace_dataset(
            monitor_data,
            name="event_time_zero",
            values=event_index.coords["event_time_zero"]
            .to(unit=monitor_data["event_time_zero"].attrs["units"], copy=False)
            .values.astype(int),
        )

In [ ]:
files = [
    "small_mcstas_sample_images.h5",
    "small_mcstas_ob_images.h5",
    "iron_simulation_sample.h5",
    "iron_simulation_ob.h5",
]

for file in files:
    mcstas_to_nexus(
        mcstas_data_file=file,
        template_nexus_file="977695_00072982.hdf",
        outfile=file.replace(".h5", ".nxs"),
        nevents=1e6,
    )